# The Short Introduction to NeuRec

## Introduction

[NeuRec](https://github.com/wubinzzu/NeuRec) is a library of recommender systems.
It was developed with a focus on enabling fast running recommendation model.
Based on NeuRec, researchers are able to go from idea to result without concern for trivia, such as data preprocessing and loading, parameter passing, data sampling and iterating, model evaluating, result recording, etc.

The main components and its brief introduction:

- `Configurator`: This class can reads arguments from an *ini-style* configuration file and parse arguments from the command line simultaneously.
The values of arguments are converted from `str` to `int`, `float`, `bool`, `list` and `None` automatically.

- `Dataset`: This class remaps the IDs of users and items to consecutive numbers, filters users and items with few interactions, and splits data with leave-one-out or fold-out (ratio) by time or randomly.

- `DataIterator`: This class combines some data sets and provides a batch iterator over them.

- `ParallelSampler`: This is an abstract class and aims to parallelize the processes of training and negative item sampling.
The parallelization and sample iteration are already implemented in
this abstract class. Every subclass only needs to provide the no argument `sampling` method.
Some commonly used samplers have been implemented, such as `PointwiseSampler`, `PairwiseSampler`, `TimeOrderPointwiseSampler` and `TimeOrderPairwiseSampler`.

- `Logger`: This is a simple encapsulation of python logging.
This class can show a message on standard output and write it into a file simultaneously.
This is convenient for observing and saving training results.

- `ProxyEvaluator`: This is an interface to evaluate the ranking performance of recommendation models.
The ranking metrics are configurable.
This class and its evaluation metrics can automatically fit both leave-one-out and fold-out splitting data without specific indication.
And the model performance can be viewed in user groups, which is activated only by an argument.
There are two implementation versions of this class: *python* and *cpp*.
**Based on the *cpp* implementation, we can fast evaluate recommendation models without sampling candidate negative items.**


## Quick Start

Firstly, download this repository and unpack the downloaded source to a suitable location.

Secondly, go to '*./NeuRec*' and compline the evaluator of cpp implementation with the following command line:

```bash
python setup.py build_ext --inplace
```

If the compilation is successful, the evaluator of cpp implementation will be called automatically.
Otherwise, the evaluator of python implementation will be called.

**Note that the cpp implementation is much faster than python.**

Thirdly, specify dataset and recommender in configuration file *NeuRec.properties*.

Finally, run [main.py](./main.py) in IDE or with command line:

```bash
python main.py
```


## An Example of Matrix Factorization Model

This example aims to describe the building blocks of NeuRec.

Following this example, researchers can fast implement their idea and conduct experiments.

### Configurator

First, we use `Configurator` to read the parameters from *NeuRec.properties*.
If there is more than one section in the configuration file, `Configurator` will read parameters from `default_section` section.

Note that, this class can also parse parameters from the command line simultaneously:

```bash
python main.py --recommender=MF --learning_rate=0.001 --batch_size=128
```

And the parameters from the command line will cover the parameters of configuration files.

Besides the *NeuRec.properties* and command line, `Configurator` also read model parameters from file '*`config_dir`/`recommender`.properties*', where `config_dir` and `recommender` are specified in *NeuRec.properties*.

In [ ]:
from util import Configurator
conf = Configurator("NeuRec.properties", default_section="hyperparameters")


The type of parameters are converted automatically.

In [ ]:
print(type(conf["data.input.dataset"]), conf["data.input.dataset"])
print(type(conf["recommender"]), conf["recommender"])
print(type(conf["learning_rate"]), conf["learning_rate"])
print(type(conf["group_view"]), conf["group_view"])
print(type(conf["topk"]), conf["topk"])


### Dataset

Dataset will be loaded and preprocessed according to the parameters in *NeuRec.properties*:

- `data.column.format` specifies the format of data: *UIRT*, *UIT*, *UIR*, *UI*, where *U*, *I*, *R* and *T* indicate user, item, rating and timestamp, respectively.

- `data.input.path` is the directory of dataset.

- `data.input.dataset` is the name of data files without extension name, which must be '*.rating*', '*.train*' or '*.test*'. More details are in [here](#data-splitting).

- `data.convert.separator` is the separator or delimiter of file columns.

- `user_min` and `item_min` control the preprocessing of dataset: the users (items) whose interactions less than `user_min` (`user_min`) will be filtered.
And then, whatever the preprocessing did or not, users and items will be remapped to consecutive numbers.

- `splitter` and `ratio` <a id="data-splitting"></a> specify the division method of the dataset.
`splitter` must be *ratio*, *loo* or *given*.
  - If `splitter` is *ratio* or *loo*, the extension of `data.input.dataset` must be '*.rating*'.
  - And if `splitter` is *ratio*, `ratio` and `1-ratio` items of each user will be divided into the training and test sets, respectively. This manner is also named fold-out.
  - If `splitter` is *loo*, one item of each user will be divided into the training and test sets, respectively. This manner is also named leave-one-out.
  - If `splitter` is *given*, it indicates that the dataset has been split, and the extensions of `data.input.dataset` must be '*.train*' and '*.test*'.

- `by_time` means that whether the items of each user are split by interacted time or not, if there is timestamp information in the dataset.

- `rec.evaluate.neg` is the number of candidate negative items for model evaluation.

  - If `rec.evaluate.neg=0`, model performance will be evaluated without negative items (i.e., with all items).
  - If `rec.evaluate.neg` is larger than `0`, negative items will be sampled for each user.
  - And if the dataset has been split (i.e., `splitter` is *given*) and candidate negative items have been provided, the file extension of candidate negative items must be '*.neg*' and its contents must be organized as follows, where `n` is the number of candidate item for each user and must be equal to `rec.evaluate.neg`.
  These contents are also separated by `data.convert.separator`.
    
    | | | | | |
    -|-|-|-|-
    user1 | item11 | item12 | ... | item1n
    user2 | item21 | item22 | ... | item2n
    user3 | item23 | item23 | ... | item3n

In [ ]:
from data.dataset import Dataset
dataset = Dataset(conf)


### Logger

Create a logger

In [ ]:
import time
import os
from util import Logger

timestamp = time.time()
data_name = dataset.dataset_name
model_name = conf["recommender"]

param_str = "%s_%s" % (data_name, conf.params_str())
run_id = "%s_%.8f" % (param_str[:150], timestamp)

log_dir = os.path.join("log", data_name, model_name)
logger_name = os.path.join(log_dir, run_id + ".log")
logger = Logger(logger_name)


Show the statistic of dataset, `logger.info()` print a message on standard output and write it into a file simultaneously.

In [ ]:
print("log file is:\t", logger_name)
logger.info(dataset)


### ProxyEvaluator

`ProxyEvaluator` is the interface to evaluate models and contains various evaluation protocols.

- Evaluation metrics are configurable via the argument `metric`.
If `metric` is `None`, metric will be set to `["Precision", "Recall", "MAP", "NDCG", "MRR"]`.
Otherwise, `metric` must be one or a sublist of metrics mentioned above.

- Evaluation metrics can automatically fit both leave-one-out and fold-out data splitting without specific indication.
In leave-one-out evaluation:
  - `Recall` is equal to `HitRatio`;
  - The implementation of `NDCG` is compatible with fold-out;
  - `MAP` and `MRR` have the same numeric values;
  - `Precision` is meaningless.

- `top_k` controls the Top-K item ranking performance. Defaults to `50`.
  - If `top_k` is an integer, K ranges from `1` to `top_k`;
  - If `top_k` is a list of integers, K are only assigned these values.

- The ranking performance of models can be viewed in user groups, which are split according to the numbers of users' interactions in training data.
This function can be activated by the argument `group_view`.
  - If `group_view` is `None`, the ranking performance will be viewed without groups.
  - If `group_view` is a list of integers, the ranking performance will be view in groups.
  - For example, if `group_view = [10,30,50,100]`, users will be split into four groups: `(0, 10]`, `(10, 30]`, `(30, 50]` and `(50, 100]`. And the users whose interacted items more than `100` will be discarded.

- **Importantly**, there are two implementation versions of this class: *python* and *cpp*.
And both of the two versions are multi-threaded.
Based on the *cpp* implementation, we can fast evaluate recommendation models without sampling candidate negative items.

Using the following command to compile cpp code:

```bash
python setup.py build_ext --inplace
```

After the successful compilation, the cpp version will be called automatically.


In [ ]:
from evaluator import ProxyEvaluator

evaluator = ProxyEvaluator(dataset.get_user_train_dict(),
                           dataset.get_user_test_dict(),
                           dataset.get_user_test_neg_dict(),
                           metric=["Precision", "NDCG"],
                           group_view=conf["group_view"],
                           top_k=[10, 20],
                           batch_size=conf["test_batch_size"],
                           num_thread=conf["num_thread"])


### Matrix factorization

In [ ]:
import tensorflow as tf


class MF(object):
    def __init__(self, dataset, conf):
        self.learning_rate = conf["learning_rate"]
        self.embedding_size = conf["embedding_size"]
        self.reg_mf = conf["reg_mf"]
        self.stddev = conf["stddev"]
        self.num_users = dataset.num_users
        self.num_items = dataset.num_items
    
    def _create_placeholder(self):
        self.user_ph = tf.placeholder(tf.int32, shape=[None])
        self.pos_item_ph = tf.placeholder(tf.int32, shape=[None])
        self.neg_item_ph = tf.placeholder(tf.int32, shape=[None])
    
    def _create_variables(self):
        initializer = tf.random_uniform_initializer(-self.stddev, self.stddev)
        self.user_embeddings = tf.Variable(initializer([self.num_users, self.embedding_size]), dtype=tf.float32)
        self.item_embeddings = tf.Variable(initializer([self.num_items, self.embedding_size]), dtype=tf.float32)
    
    def build_model(self):
        self._create_placeholder()
        self._create_variables()
        user_emb = tf.nn.embedding_lookup(self.user_embeddings, self.user_ph)
        pos_item_emb = tf.nn.embedding_lookup(self.item_embeddings, self.pos_item_ph)
        neg_item_emb = tf.nn.embedding_lookup(self.item_embeddings, self.neg_item_ph)

        pos_rating = tf.reduce_sum(tf.multiply(user_emb, pos_item_emb), 1)
        neg_rating = tf.reduce_sum(tf.multiply(user_emb, neg_item_emb), 1)
        
        # ranking loss
        ranking_loss = -tf.reduce_sum(tf.log_sigmoid(pos_rating-neg_rating))
        
        # reg loss
        params = [user_emb, pos_item_emb, neg_item_emb]
        reg_loss = tf.add_n([tf.nn.l2_loss(w) for w in params])

        final_loss = ranking_loss + self.reg_mf*reg_loss
        
        self.update_opt = tf.train.AdamOptimizer(self.learning_rate).minimize(final_loss)

        # for evaluating model
        self.predict_ratings = tf.matmul(user_emb, self.item_embeddings, transpose_b=True)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def train_step(self, users, pos_items, neg_items):
        feed_dict = {self.user_ph: users, 
                     self.pos_item_ph: pos_items, 
                     self.neg_item_ph: neg_items}
        self.sess.run(self.update_opt, feed_dict=feed_dict)

    def predict(self, users, candidate_items=None):
        # evaluator will call this method to get item ranking scores 
        # and evaluate model performance
        ratings = self.sess.run(self.predict_ratings, feed_dict={self.user_ph: users})
        return ratings


### ParallelSampler

In [ ]:
from data import PairwiseSampler

data_sampler = PairwiseSampler(dataset, neg_num=1, batch_size=conf["batch_size"], shuffle=True)


### Train model

In [ ]:
model = MF(dataset, conf)
model.build_model()

num_epochs = conf["epochs"]
batch_size = conf["batch_size"]

logger.info(conf)

logger.info(evaluator.metrics_info())
for epoch in range(num_epochs):
    for bat_users, bat_items_pos, bat_items_neg in data_sampler:
        model.train_step(bat_users, bat_items_pos, bat_items_neg)
    logger.info("epoch: %d:\t%s"%(epoch, evaluator.evaluate(model)))
